# PEFT Project
simple project to demonstrate Parameter-Efficient Fine-Tunning Model using PEFT Library

Install needed packages

In [1]:
%pip install --index-url https://download.pytorch.org/whl/cu128 torch torchvision torchaudio
%pip install transformers peft scikit-learn pandas huggingface_hub datasets hf_xet evaluate ipywidgets

Looking in indexes: https://download.pytorch.org/whl/cu128
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# import needed classes

In [2]:
import torch
from transformers import GPT2ForSequenceClassification,AutoTokenizer,DataCollatorWithPadding,TrainingArguments,Trainer,pipeline
from datasets import load_dataset
import evaluate
import numpy as np
from peft import LoraConfig, get_peft_model, PeftModel,TaskType,AutoPeftModelForCausalLM,PeftModelForSequenceClassification,AutoPeftModelForSequenceClassification

## Helper functions

In [3]:
def get_gpt2_model_and_tokenizer():
    """
    Load GPT2 pre-trained model from HugginFace
    Create and Configure Tokenizer
    return model and tokenizer
    """
    base_model_name = 'gpt2'
    device = "cuda" if torch.cuda.is_available() else "cpu"
    base_model = GPT2ForSequenceClassification.from_pretrained(
        base_model_name,
        num_labels=2,
        id2label={0:"neg",1:"pos"},
        label2id={"neg":0,"pos":1}
    )
    base_model = base_model.to(device)
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    # GPT does not have padding by default and cause error with padding
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token  
    tokenizer.padding_side = "left"
    base_model.config.pad_token_id = tokenizer.pad_token_id
    return base_model, tokenizer

## Load pre-trained model and tokenizer

In [4]:
# load pre_trained model
base_model,tokenizer = get_gpt2_model_and_tokenizer()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Test model manually

In [5]:
tests = [
    "I feel little rainy ",
    "there are many definitions of 'time waster",
    "movie has good music"
]

base_model_pipeline = pipeline("text-classification", model=base_model, tokenizer=tokenizer, top_k=1)
for test in tests:
    print(f"base model test with '{test}' result => {base_model_pipeline(test)}")

Device set to use cuda:0


base model test with 'I feel little rainy ' result => [[{'label': 'neg', 'score': 0.9875401854515076}]]
base model test with 'there are many definitions of 'time waster' result => [[{'label': 'neg', 'score': 0.959682285785675}]]
base model test with 'movie has good music' result => [[{'label': 'neg', 'score': 0.9945533275604248}]]


## Datasets

In [6]:
# load dataset
dataset_name = "imdb" # "dair-ai/emotion" # classify text emotion
dataset = load_dataset(dataset_name) 
print(dataset)
# define tokenizer function
def tokenize_fun(dataset):
    return tokenizer(dataset["text"], padding="max_length", truncation=True)

# define datasets
train_dataset = dataset["train"].select(range(100)).map(tokenize_fun, batched=True) # limit the dataset for fast training
test_dataset = dataset["test"].select(range(50)).map(tokenize_fun, batched=True) # limit the dataset for fast evaluation

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

## Evaluate pre-trained model

In [7]:
#define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# test pre-trained model against dataset and note total loss
data_collator = DataCollatorWithPadding(tokenizer=tokenizer,pad_to_multiple_of=8)
accuracy = evaluate.load("accuracy")

# start evaluation 
original_eval_output_dir = "original_eval_out"
original_model_eval_args = TrainingArguments(
    output_dir= original_eval_output_dir,
    do_train=False, # don't train the model
    do_eval=True,
    per_device_eval_batch_size=64,
    dataloader_num_workers=0, # cause issue on windows
    fp16=torch.cuda.is_available(),
    report_to=[]
)

original_trainer = Trainer(
    model=base_model,
    args=original_model_eval_args,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
original_metrics = original_trainer.evaluate()
print(original_metrics)

C:\Users\engmu\AppData\Local\Temp\ipykernel_3156\1346320767.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  original_trainer = Trainer(


{'eval_loss': 0.06124982237815857, 'eval_model_preparation_time': 0.0015, 'eval_accuracy': 1.0, 'eval_runtime': 2.0733, 'eval_samples_per_second': 24.116, 'eval_steps_per_second': 0.482}


## Create PEFT model from pre-trained one

In [ ]:
# create peft model
print(base_model)
config = LoraConfig(
    task_type=TaskType.SEQ_CLS,    
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["c_attn", "c_proj", "c_fc"],  
    modules_to_save=["score"],    
)

peft_original_model, peft_tokenizer = get_gpt2_model_and_tokenizer()

peft_model = get_peft_model(peft_original_model,config)
peft_model.print_trainable_parameters()

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,181,184 || all params: 125,622,528 || trainable%: 0.9403


c:\Users\engmu\projects\learn\udacity-genai-peft\.venv\Lib\site-packages\peft\tuners\lora\layer.py:2156: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


## Train PEFT Model on train dataset

In [9]:
# train the model against dataset
training_model_output_dir="peft_gpt_train_out"
peft_model_train_args = TrainingArguments(
    output_dir=training_model_output_dir,
    do_train=True,
    do_eval=False,
    per_device_train_batch_size=64,
    dataloader_num_workers=0,
    num_train_epochs=3,
    fp16=torch.cuda.is_available(),
    report_to=[],
    group_by_length=True
)

peft_trainer = Trainer(
    model=peft_model,
    args=original_model_eval_args,
    train_dataset=train_dataset,
    tokenizer=peft_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
metrics = peft_trainer.train()
print(metrics)
peft_model.save_pretrained("gpt2-mustafa")


C:\Users\engmu\AppData\Local\Temp\ipykernel_3156\2246569190.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  peft_trainer = Trainer(


Step,Training Loss


TrainOutput(global_step=39, training_loss=0.28437379690317005, metrics={'train_runtime': 199.1629, 'train_samples_per_second': 1.506, 'train_steps_per_second': 0.196, 'total_flos': 158955208704000.0, 'train_loss': 0.28437379690317005, 'epoch': 3.0})


# Test PEFT Model

In [10]:
peft_model = AutoPeftModelForSequenceClassification.from_pretrained(
    'gpt2-mustafa',
    num_labels=2,
    id2label={0:"neg",1:"pos"},
    label2id={"neg":0,"pos":1}
)
peft_model.config.pad_token_id = peft_tokenizer.pad_token_id
peft_eval_output_dir = "peft_eval_out"
peft_model_eval_args = TrainingArguments(
    output_dir=peft_eval_output_dir,
    do_train=False,
    do_eval=True,
    per_device_eval_batch_size=32,
    dataloader_num_workers=0,
    fp16=torch.cuda.is_available(),
    report_to=[]
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_model_eval_args,
    eval_dataset=test_dataset,
    tokenizer=peft_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
peft_metrics = peft_trainer.evaluate()
print(peft_metrics)
print(metrics)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\engmu\AppData\Local\Temp\ipykernel_3156\985751935.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  peft_trainer = Trainer(


{'eval_loss': 0.03673555329442024, 'eval_model_preparation_time': 0.0135, 'eval_accuracy': 1.0, 'eval_runtime': 31.1127, 'eval_samples_per_second': 1.607, 'eval_steps_per_second': 0.064}
TrainOutput(global_step=39, training_loss=0.28437379690317005, metrics={'train_runtime': 199.1629, 'train_samples_per_second': 1.506, 'train_steps_per_second': 0.196, 'total_flos': 158955208704000.0, 'train_loss': 0.28437379690317005, 'epoch': 3.0})


## Compare between original model and PEFT model

In [11]:
print(f"peft result: original model eval loss {original_metrics["eval_loss"]} => peft model eval loss {peft_metrics["eval_loss"]}")

peft result: original model eval loss 0.06124982237815857 => peft model eval loss 0.03673555329442024


## manually test both original model and PEFT model

In [12]:
tests = [
    "I feel little rainy ",
    "there are many definitions of 'time waster",
    "movie has good music"
]


peft_model_pipeline = pipeline("text-classification", model=peft_model,tokenizer=tokenizer, top_k=1)
for test in tests:
    print(f"peft model test with '{test}' result => {peft_model_pipeline(test)}")

print(base_model)
print(peft_original_model)

Device set to use cuda:0


peft model test with 'I feel little rainy ' result => [[{'label': 'neg', 'score': 0.6095241904258728}]]
peft model test with 'there are many definitions of 'time waster' result => [[{'label': 'neg', 'score': 0.5869964361190796}]]
peft model test with 'movie has good music' result => [[{'label': 'neg', 'score': 0.7563575506210327}]]
GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
  